## Step 1: Load the Data from Excel

In [27]:
import pandas as pd

# Load the data from the Excel file
file_path = '/content/drive/MyDrive/Colab Notebooks/Fiverr/Ranuja/customer_recommendation/Ashleybah OpenOrder 1.xlsx'
df = pd.read_excel(file_path)
df

,company_Name,Store_Name,Model ID,DESCRIPTION,VCost,BasePrice,Cubes,ONH(Unit/CTN),RES,Availble(W),...,2024-01-01 00:00:00,Total Po of Units,POJun-2024,POMay-2024,POApr-2024,POMar-2024,POFeb-2024,POJan-2024,Monthly Average,# Month of Supply
0,Ashley-BAH,NaN,NaN,NaN,NaN,NaN,NaN,8878 (1031),1136,8615,...,855,16600,1250,1296,1029,1368,4204,7455,924.17,2766.67
1,Ashley-BAH,Nassau,Nassau,NaN,NaN,NaN,NaN,8878 (1031),1136,8615,...,855,16600,1250,1296,1029,1368,4204,7455,924.17,2766.67
2,Ashley-BAH,Nassau,A2000477,Basket (2/CN),58.20,130.99,3.33,9 (5),0,9,...,1,16,0,0,0,2,4,10,0.50,2.67
3,Ashley-BAH,Nassau,A52000337,TRAY (2/CS),76.00,239.40,0.00,9 (5),0,9,...,0,10,0,0,0,0,10,0,0.17,1.67
4,Ashley-BAH,Nassau,A1000912,Throw (3/CS),75.00,188.97,0.00,9 (3),0,9,...,0,12,0,0,0,0,12,0,0.50,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920,Ashley-BAH,Nassau,B799-46,FIVE DRAWER CHEST,465.60,1439.99,25.56,0,1,0,...,1,5,1,0,1,0,0,3,1.17,0.83
1921,Ashley-BAH,Nassau,P305-050,CHAIRS W/CUSH/TABLE SET (3/CN),203.70,577.99,13.22,0,0,0,...,4,13,0,0,0,1,9,3,2.17,2.17
1922,Ashley-BAH,Nassau,B740-56,KING/CAL KING PANEL FOOTBOARD,140.00,440.99,4.74,0,1,0,...,0,2,0,0,0,0,0,2,0.50,0.33
1923,Ashley-BAH,Nassau,A1000838,Pillow (4/CS),50.44,175.96,1.13,0,0,0,...,0,12,0,0,0,0,0,12,1.67,2.00


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1925 entries, 0 to 1924
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_Name            1925 non-null   object 
 1   Store_Name              1924 non-null   object 
 2   Model ID                1924 non-null   object 
 3   DESCRIPTION             1923 non-null   object 
 4   VCost                   1923 non-null   float64
 5   BasePrice               1923 non-null   float64
 6   Cubes                   1923 non-null   float64
 7   ONH(Unit/CTN)           1925 non-null   object 
 8   RES                     1925 non-null   int64  
 9   Availble(W)             1925 non-null   int64  
 10  POs                     1925 non-null   int64  
 11  QTY                     1925 non-null   int64  
 12  Availble(S)             1769 non-null   float64
 13  #Floor                  1769 non-null   float64
 14  Total No of Units sold  1925 non-null   

## Step 2: Preprocess the Data

In [28]:
# Select the columns using indices
columns_to_keep = df.columns[[2, 3, 15, 16, 17, 18, 19, 20]]
df = df[columns_to_keep]

df = df.dropna()

# Convert to long format
df_long = pd.melt(df, id_vars=['Model ID', 'DESCRIPTION'], var_name='Month', value_name='Units Sold')

# Convert 'Month' to datetime
df_long['Month'] = pd.to_datetime(df_long['Month'])

# Sort the dataframe by 'Model ID' and 'Month'
df_long = df_long.sort_values(by=['Model ID', 'Month'])
df_long

,Model ID,DESCRIPTION,Month,Units Sold
10977,1020208,OVERSIZED ACCENT OTTOMAN,2024-01-01,0
9054,1020208,OVERSIZED ACCENT OTTOMAN,2024-02-01,0
7131,1020208,OVERSIZED ACCENT OTTOMAN,2024-03-01,1
5208,1020208,OVERSIZED ACCENT OTTOMAN,2024-04-01,0
3285,1020208,OVERSIZED ACCENT OTTOMAN,2024-05-01,1
...,...,...,...,...
8510,W796-68,XL TV STAND W/FIREPLACE OPTION,2024-02-01,2
6587,W796-68,XL TV STAND W/FIREPLACE OPTION,2024-03-01,1
4664,W796-68,XL TV STAND W/FIREPLACE OPTION,2024-04-01,0
2741,W796-68,XL TV STAND W/FIREPLACE OPTION,2024-05-01,1


## Step 3: Model Training and Forecasting

In [23]:
!pip install prophet

In [29]:
from prophet import Prophet

# Forecast for each product
forecasts = {}
for model_id, group in df_long.groupby('Model ID'):
    # Prepare data for Prophet
    prophet_df = group[['Month', 'Units Sold']].rename(columns={'Month': 'ds', 'Units Sold': 'y'})

    # Initialize and fit the model
    model = Prophet()
    model.fit(prophet_df)

    # Make future dataframe for the next month
    future = model.make_future_dataframe(periods=1, freq='M')
    forecast = model.predict(future)

    # Store forecast
    forecasts[model_id] = forecast[['ds', 'yhat']].tail(1)

# Combine forecasts into a single DataFrame
forecast_results = pd.concat(forecasts).reset_index().drop('level_1', axis=1)
forecast_results.columns = ['Model ID', 'Date', 'Predicted Units Sold']

print(forecast_results)


Streaming output truncated to the last 5000 lines.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 3.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcp4y298l/pj_m41n6.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpcp4y298l/zuxhnjlp.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=97098', 'data', 'file=/tmp/tmpcp4y298l/pj_m41n6.json', 'init=/tmp/tmpcp4y298l/zuxhnjlp.json', 'output', 'file=/tmp/tmpcp4y298l/prophet_modelvs_upse0/prophet_model-20240710124150.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
12:41:50 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:41:50 - 

      Model ID       Date  Predicted Units Sold
0      1020208 2024-06-30              0.520689
1      1020246 2024-06-30              0.213433
2      1020264 2024-06-30             -0.445369
3      1020265 2024-06-30             -0.054502
4      1020277 2024-06-30             -0.054502
...        ...        ...                   ...
1918  U5950415 2024-06-30              1.208934
1919  U5950418 2024-06-30              1.009686
1920  U5950513 2024-06-30              0.659959
1921  W446-168 2024-06-30              0.265295
1922   W796-68 2024-06-30              1.609568

[1923 rows x 3 columns]


## Forecasting for the next month

In [32]:
forecast_results

,Model ID,Date,Predicted Units Sold
0,1020208,2024-06-30,0.520689
1,1020246,2024-06-30,0.213433
2,1020264,2024-06-30,-0.445369
3,1020265,2024-06-30,-0.054502
4,1020277,2024-06-30,-0.054502
...,...,...,...
1918,U5950415,2024-06-30,1.208934
1919,U5950418,2024-06-30,1.009686
1920,U5950513,2024-06-30,0.659959
1921,W446-168,2024-06-30,0.265295


In [37]:
# prompt: create a column by rounding off

forecast_results['Rounded Predicted Units Sold(Next Month)'] = forecast_results['Predicted Units Sold'].round()
forecast_results


,Model ID,Date,Predicted Units Sold,Rounded Predicted Units Sold(Next Month)
0,1020208,2024-06-30,0.520689,1.0
1,1020246,2024-06-30,0.213433,0.0
2,1020264,2024-06-30,-0.445369,-0.0
3,1020265,2024-06-30,-0.054502,-0.0
4,1020277,2024-06-30,-0.054502,-0.0
...,...,...,...,...
1918,U5950415,2024-06-30,1.208934,1.0
1919,U5950418,2024-06-30,1.009686,1.0
1920,U5950513,2024-06-30,0.659959,1.0
1921,W446-168,2024-06-30,0.265295,0.0


In [39]:
forecast_results.to_excel('forecast_results.xlsx', index=False)